# Agent Development Kit integration with Google Login

This notebook demonstrates a working ADK agent with integrated Google login.

**Key highlights**:
- The agent proactively requests authentication when necessary, directing the application to the Google login screen.
- An encrypted token is stored in the session state and reused by the agent.
- Client secrets and the encryption key are securely managed within Secret Manager.

The implementation follows the official guide.
- [Authenticating with Tools](https://google.github.io/adk-docs/tools/authentication/)

**Note**:
The client secrets shown within this notebook belong to a non-existent project and cannot actually be used.

## Preparation

1. Enable APIs (Run on Cloud Shell)
```bash
gcloud services enable \
  aiplatform.googleapis.com \
  notebooks.googleapis.com \
  cloudresourcemanager.googleapis.com \
  calendar-json.googleapis.com \
  secretmanager.googleapis.com
```

2. Add `secretmanager.secretAccessor` role to Compute Engine default service account. (Run on Cloud Shell)
```bash
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
PROJECT_NUMBER=$(gcloud projects describe $PROJECT_ID --format='value(projectNumber)')
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:$PROJECT_NUMBER-compute@developer.gserviceaccount.com" \
    --role='roles/secretmanager.secretAccessor'
```

3. Create Vertex AI Workbench instance (Run on Cloud Shell)
```bash
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
gcloud workbench instances create agent-development \
  --project=$PROJECT_ID \
  --location=us-central1-a \
  --machine-type=e2-standard-2
```

4. [Configure the OAuth consent screen and choose scopes](https://developers.google.com/workspace/guides/configure-oauth-consent)
  - Select Internal user type.
 
5. [Configure OAuth client ID credentials](https://developers.google.com/workspace/guides/create-credentials#oauth-client-id)
  - Create "Web application" client and add `https://www.example.com` to the authorized redirect URI.
  - Download the client secret JSON file as `credentials.json`

6. Navigate to [Workbench Instances](https://console.cloud.google.com/vertex-ai/workbench/instances) on Cloud Console and open [JupyterLab].
  - Open a new "Python 3(ipykernel)" notebook.
  - Upload `credentials.json` to the current directory.
  
Now you can follow the notebook contents below.

In [ ]:
%pip install --upgrade --user google-adk

In [4]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Configure secret manager

Import modules and set environment variables.

In [1]:
import json, os
import vertexai
from cryptography.fernet import Fernet
from google.cloud import secretmanager

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=f'gs://{PROJECT_ID}'
)

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

Helper functions to use secret manager.

In [2]:
def create_serect(secret_id):
    client = secretmanager.SecretManagerServiceClient()
    response = client.create_secret(
        request={
            'parent': f'projects/{PROJECT_ID}',
            'secret_id': secret_id,
            'secret': {'replication': {'automatic': {}}},
        }
    )
    return response.name

def add_secret_version(secret_name, secret_value):
    client = secretmanager.SecretManagerServiceClient()
    if isinstance(secret_value, str):
        payload = secret_value.encode('utf-8')
    else:
        payload = secret_value
    response = client.add_secret_version(
        request={'parent': secret_name, 'payload': {'data': payload}}
    )
    return response.name

Create a secret `client_credentials` to store client credentials (`credentials.json`).

In [3]:
client_credentials_name = create_serect('client_credentials')
client_credentials_name

'projects/362574666120/secrets/client_credentials'

Create a secret `token_encryption_key` to store an encryption key
that will be used to encrypt / decrypt access tokens.

In [4]:
token_encryption_key_name = create_serect('token_encryption_key')
token_encryption_key_name

'projects/362574666120/secrets/token_encryption_key'

Store `credentials.json` in `client_credentials` secret.

In [5]:
with open('credentials.json', 'rt') as f:
    client_credentials = f.read()
add_secret_version(client_credentials_name, client_credentials)

'projects/362574666120/secrets/client_credentials/versions/1'

Create and store an encryption key in `token_encryption_key` secret.

In [6]:
key = Fernet.generate_key()
add_secret_version(token_encryption_key_name, key)

'projects/362574666120/secrets/token_encryption_key/versions/1'

Define helper functions to use secrets.

In [7]:
def get_client_credentials():
    client = secretmanager.SecretManagerServiceClient()
    name = f'projects/{PROJECT_ID}/secrets/client_credentials/versions/latest'
    response = client.access_secret_version(name=name)
    return json.loads(response.payload.data.decode('utf-8'))

def get_encryption_key():
    client = secretmanager.SecretManagerServiceClient()
    name = f'projects/{PROJECT_ID}/secrets/token_encryption_key/versions/latest'
    response = client.access_secret_version(name=name)
    return response.payload.data

def encrypt_token(token: str) -> str:
    f = Fernet(get_encryption_key())
    encrypted_token = f.encrypt(token.encode()).decode()
    return encrypted_token

def decrypt_token(encrypted_token: str) -> str:
    f = Fernet(get_encryption_key())
    decrypted_token = f.decrypt(encrypted_token.encode()).decode()
    return decrypted_token

In [8]:
get_client_credentials()['web']

{'client_id': '__masked__.apps.googleusercontent.com',
 'project_id': '__masked__',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'token_uri': 'https://oauth2.googleapis.com/token',
 'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'client_secret': '__masked__',
 'redirect_uris': ['https://www.example.com']}

## Helper functions to handle authentication during a session

In [9]:
from fastapi.openapi.models import (
    OAuth2, OAuthFlowAuthorizationCode, OAuthFlows
)
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google.adk.auth import (
    AuthCredential, AuthCredentialTypes,
    OAuth2Auth, AuthConfig
)

from google.genai.types import (
    FunctionResponse, Part, UserContent
)
from google.adk.events import Event, EventActions
from google.adk.tools import ToolContext
from google.adk.agents.llm_agent import LlmAgent
from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService

In [10]:
client_creds = get_client_credentials()['web']

auth_scheme = OAuth2(
    flows=OAuthFlows(
        authorizationCode=OAuthFlowAuthorizationCode(
            authorizationUrl='https://accounts.google.com/o/oauth2/auth',
            tokenUrl='https://oauth2.googleapis.com/token',
            scopes={
                'https://www.googleapis.com/auth/calendar': 'calendar scope'
            },
        )
    )
)

auth_credential = AuthCredential(
    auth_type=AuthCredentialTypes.OAUTH2,
    oauth2=OAuth2Auth(
        client_id=client_creds['client_id'],
        client_secret=client_creds['client_secret'],
    ),
)

Function to get a credential using the token stored in the session state.

In [37]:
def get_credential(tool_context, scopes):
    TOKEN_CACHE_KEY = 'encripted_token'
    creds = None
    cached_encripted_token = tool_context.state.get(TOKEN_CACHE_KEY)
    
    if cached_encripted_token: # Token is stored in the session state.
        token_info = json.loads(decrypt_token(cached_encripted_token))
        # Construct a credential from the token, and refresh the token if necessary.
        try:
            creds = Credentials.from_authorized_user_info(token_info, scopes)
            if not creds.valid:
                if creds.expired and creds.refresh_token:
                    print('## Refresh token')
                    creds.refresh(Request())
                    tool_context.state[TOKEN_CACHE_KEY] = encrypt_token(creds.to_json())
                else:
                    creds = None
                    tool_context.state[TOKEN_CACHE_KEY] = None

        except Exception as e:
            print(f'## Error loading/refreshing cached creds: {e}')
            creds = None
            tool_context.state[TOKEN_CACHE_KEY] = None

    # If a credential is not available...,
    if not (creds and creds.valid): 
        # Check if the user has sent an auth response to the agent.
        exchanged_credential = tool_context.get_auth_response(AuthConfig(
          auth_scheme=auth_scheme,
          raw_auth_credential=auth_credential,
        ))
        # If not, retrun 'adk_request_credential' message as a function call.
        if not exchanged_credential:
            tool_context.request_credential(AuthConfig(
                auth_scheme=auth_scheme,
                raw_auth_credential=auth_credential,
            ))
            return None
    
        # Receive a new token using exchanged_credential that the user has sent.
        access_token = exchanged_credential.oauth2.access_token
        refresh_token = exchanged_credential.oauth2.refresh_token
        creds = Credentials(
            token=access_token,
            refresh_token=refresh_token,
            token_uri=auth_scheme.flows.authorizationCode.tokenUrl,
            client_id=auth_credential.oauth2.client_id,
            client_secret=auth_credential.oauth2.client_secret,
            scopes=list(auth_scheme.flows.authorizationCode.scopes.keys()),
        )        
        tool_context.state[TOKEN_CACHE_KEY] = encrypt_token(creds.to_json())

    # Now the token is stored in the session state.
    # Recreate a credential with specified scopes.
    cached_encripted_token = tool_context.state.get(TOKEN_CACHE_KEY)
    token_info = json.loads(decrypt_token(cached_encripted_token))
    creds = Credentials.from_authorized_user_info(token_info, scopes)  
    return creds

Function to check the 'adk_request_credential' message in events from an agent.

In [38]:
def check_pending_auth_event(event: Event) -> (str, AuthConfig):
    if not (event.content and event.content.parts):
        return None, None
    for part in event.content.parts:
        if not part.function_call:
            continue
        if part.function_call.name == 'adk_request_credential':
            function_call_id = part.function_call.id
            auth_config = AuthConfig(
                **part.function_call.args.get('auth_config')
            )
            return function_call_id, auth_config
    return None, None

## Tool function example that requries Google authentication

In [39]:
import datetime
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

def my_authenticated_tool_function(tool_context: ToolContext) -> dict:
    ''' Get upcoming events from Google Calendar '''

    scopes = ['https://www.googleapis.com/auth/calendar']
    # Get a credential using the helper function
    creds = get_credential(tool_context, scopes) 
    # If a credential is not available, retrun a pending messsage.
    if not creds:
        return {'pending': True, 'message': 'Awaiting user authentication.'}
    
    try:
        service = build('calendar', 'v3', credentials=creds)
        now = datetime.datetime.now(datetime.timezone.utc).strftime('%Y-%m-%dT%H:%M:%S%z')
        now = now[:-2] + ':' + now[-2:] # Format to include colon in timezone offset
        events_result = (
            service.events().list(
                calendarId='primary',
                timeMin=now,
                maxResults=10,
                singleEvents=True,
                orderBy='startTime',
            ).execute()
        )
        events = events_result.get('items', [])
        return {'events': events}

    except HttpError as error:
        return {'error': error}

## Example use of an agent with the tool function

Define an agent with my_authenticated_tool_function.

In [40]:
agent = LlmAgent(
    model='gemini-2.0-flash-001',
    name='upcoming_events',
    description=(
        'This agent gets upcoming events.'
    ),
    instruction='''
If the user asks to get upcoming events, use my_authenticated_tool_function()
Show the result in a human readable format using an ascii table.
Headers are Summary, Start Time, End Time
''',
    tools=[
        my_authenticated_tool_function
    ],
)

Function to emulate google login on the notebook.

In [41]:
import asyncio

async def auth_request(auth_request_event_id, auth_config):
    redirect_uri = 'https://www.example.com'
    auth_request_uri = (
        auth_config.exchanged_auth_credential.oauth2.auth_uri
        + f'&redirect_uri={redirect_uri}'
    )
    print('\n--- User Action Required ---')
    prompt = (
      f'1. Please open this URL in your browser to log in:\n   {auth_request_uri}\n\n'
      f'2. After successful login and authorization, your browser will be redirected.\n'
      f'   Copy the *entire* URL from the browser\'s address bar.\n\n'
      f'3. Paste the copied URL here and press Enter:\n\n> '
    )
    loop = asyncio.get_event_loop()
    auth_response_uri = await loop.run_in_executor(None, input, prompt)
    auth_config.exchanged_auth_credential.oauth2.auth_response_uri = auth_response_uri
    auth_config.exchanged_auth_credential.oauth2.redirect_uri = redirect_uri
    auth_content = UserContent(
      parts=[
          Part(
              function_response=FunctionResponse(
                  id=auth_request_event_id,
                  name='adk_request_credential',
                  response=auth_config.model_dump(),
              )
          )
      ],
    )
    return auth_content

Local chat application with a google login feature.

In [42]:
async def local_app(user_id='user'):
    runner = Runner(
      app_name='my_app',
      agent=agent,
      artifact_service=InMemoryArtifactService(),
      session_service=session_service,
    )
    
    query = 'Get upcoming events.'
    print(f'[user]\n{query}')
    content = UserContent(parts=[Part.from_text(text=query)])
    
    while content:
        events_async = runner.run_async(
            session_id=session.id, user_id=user_id, new_message=content
        )
        content = None
        auth_request_event_id = None
        async for event in events_async:
            print(f'----\n{event}\n----')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(f'[agent]\n{response}')
            if not auth_request_event_id:
                auth_request_event_id, auth_config = check_pending_auth_event(event)
        
        if auth_request_event_id:
            print('--> Authentication required by agent.')
            content = await auth_request(auth_request_event_id, auth_config)

Store a session service in a global variable so that we can check the state contents before / after agent execution.

In [45]:
session_service = InMemorySessionService()
session = session_service.create_session(
    state={},
    app_name='my_app',
    user_id='user',
    session_id='1'
)

# Confirm that the state is empty.
session_service.get_session(
    session_id='1', app_name='my_app', user_id='user'
).state

{}

For the first run, authentication is required.

In [46]:
await local_app()

[user]
Get upcoming events.


----
content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id='adk-66117524-10e4-4ef6-8ff2-10ddf9e59869', args={}, name='my_authenticated_tool_function'), function_response=None, inline_data=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None invocation_id='e-139e0889-e49e-4b6f-90ae-42e3b771de37' author='upcoming_events' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=set() branch=None id='a9uJ4jfM' timestamp=1745968093.572497
----
----
content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id='adk-6f06a044-77e9-420d-a2cc-5e3df81b0ad0', args={'function_call_id

1. Please open this URL in your browser to log in:
   https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=__masked__.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=yyXm3I7N7NudEPnfebQ7Nv99aCOWPe&access_type=offline&prompt=consent&redirect_uri=https://www.example.com

2. After successful login and authorization, your browser will be redirected.
   Copy the *entire* URL from the browser's address bar.

3. Paste the copied URL here and press Enter:

>  https://www.example.com/?state=yyXm3I7N7NudEPnfebQ7Nv99aCOWPe&code=4/0Ab_5qlkgPbsoTQlKqJ74MvAowVy7RlVWy_-Lo4uv8KqpHf7-Qb4sEiHbKPpFhWhSjRpHHw&scope=https://www.googleapis.com/auth/calendar


----
content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=FunctionResponse(id='adk-66117524-10e4-4ef6-8ff2-10ddf9e59869', name='my_authenticated_tool_function', response={'events': [{'kind': 'calendar#event', 'etag': '"3491933258751198"', 'id': '6tug384egghr21dscl5q57uttc', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=NnR1ZzM4NGVnZ2hyMjFkc2NsNXE1N3V0dGMgYWRtaW5AZW5ha2FpLmFsdG9zdHJhdC5jb20', 'created': '2025-04-27T03:00:09.000Z', 'updated': '2025-04-29T22:43:49.375Z', 'summary': 'Lunch meeting with hiroki', 'creator': {'email': 'admin@enakai.altostrat.com', 'self': True}, 'organizer': {'email': 'admin@enakai.altostrat.com', 'self': True}, 'start': {'dateTime': '2025-05-01T12:00:00+09:00', 'timeZone': 'Asia/Tokyo'}, 'end': {'dateTime': '2025-05-01T14:00:00+09:00', 'timeZone': 'Asia/Tokyo'}, 'iCalUID': '6tug384egghr21dscl5q57uttc@google.com', '

Ecnrypted token is stored in the session state.

In [47]:
session_service.get_session(
    session_id='1', app_name='my_app', user_id='user'
).state

{'encripted_token': 'gAAAAABoEVvvRCHqfPc_TEygrV2pG4CEdveKURAOQF0e9JBUqCxbL0CCOChLklfsvN8t31fnMOkk4pHwBOHVr0uRsxqvk9iwNPrwkFK9at03bma8ZpNQuPDk_NQhwKeWYSh2lFONa2egaTZkuxMV5Iyz46p1hAwR2FhGDqC7AMQ8AIgSPHPju54rGFUb2vC77PpyZHQ9dQ65zlUBXvL_nQ1jBC8zFHttR5urUu34YIn8HKiZOkPK50YB5Rxke5V3-JFX-9CGwqk7qURxtwVmEcC0T7WbmYS5UpXjpKQ0wsL_a2GRyUKgGJAb4vP8RHYzZNQRoBUQp6o5-H3f6rBHBc9tpwQozFyF8GehRLzZNjoYWE_eGv-EcXosy2rutlgXvdjSnRlqhHtAUNoIywGVML9_T2_ZoaNHPC0xseav1lbvffycNyYeMt9mt7l6BZgNIP1P_mM9_05ReI_xq2wMQ26gn8uHkY2joa93Ozq5cxBls9q8XVK2r5tXon2MLM7eag6hr6tbse0bpbskywdNN-ZwVhtQYsghC3mBXaufgmLlbl8flTck6xuECvlmYm2fmkdQjPDH3VnJoNjNkaf4RV9bVZsXpUY7yNMosdZJcT0USF7c7JFrhT2Xdepm3xNF4LKVjftWovUKq04dQpcSGWeVt3ZtzNs9Ta4Uu-XZSZz_DMY-3P_KTXNgCKcos2PglVFkWnuXmFxJMtqmwmLsSTqvL3r2H01HxY7weX1n4cNehX8VN1VUsFQCIRS5ex1v8Z7hZ14ri_YY-EC_TC5CYRF6HKOpblbZNFBxm0Mn85mGWxfUv6Ta-qXrQnT1WhxVWEaASlA_f1ZudiMZ-555g6j9phyhbuthk1niKBQke7nTzigSMAsrhZMMhW6_TamZpFOWamHVgVRuNm1Q2463F7dLt3v6D1xXkSugRbmBR11hh-NZh-H6-VY-GLXwJlliiOvIdmieEHIx'}

For the second run, no authtication is required.

In [48]:
await local_app()

[user]
Get upcoming events.


----
content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id='adk-40a413f0-e0d2-4821-83b9-e5fd26b3e71f', args={}, name='my_authenticated_tool_function'), function_response=None, inline_data=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None invocation_id='e-5475ad72-b2e7-4a28-b7c0-f5409a67af5e' author='upcoming_events' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=set() branch=None id='3ZATM76s' timestamp=1745968118.039159
----
## Refresh token
----
content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=FunctionResponse(id='adk-40a413f0-e0d2-482